## Preflight Check

If this fails with 401, regenerate OpenRouter key and restart kernel.


In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv(dotenv_path=".env", override=False)

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"].strip(),
    default_headers={
        "HTTP-Referer": "http://localhost:8888",
        "X-Title": "Dallas Agent Workshop",
    },
)

resp = client.chat.completions.create(
    model=os.getenv("OPENROUTER_MODEL", "arcee-ai/trinity-large-preview:free"),
    messages=[{"role": "user", "content": "Reply with exactly: MODEL WORKING"}],
)

print(resp.choices[0].message.content)


MODEL WORKING


# Dallas AI — Hands-on Agent Building (LangGraph + OpenRouter)

This notebook is the main workshop surface:
- You will run the agent locally.
- The model is accessed via OpenRouter.
- The agent can generate Python code and execute it using a controlled tool.

**Goal:** experience a real *plan → code → execute → fix* loop.


## 0) Setup

1. Create a venv and install deps:
```bash
pip install -r requirements.txt
```
2. Copy `.env.example` to `.env` and set `OPENROUTER_API_KEY`.
3. Restart kernel after editing `.env`.


In [2]:
import os
os.environ["OPENROUTER_MODEL"] = "arcee-ai/trinity-large-preview:free"

k = os.environ["OPENROUTER_API_KEY"]
print("repr:", repr(k[-10:]))
print("endswith newline?", k.endswith("\n"))
print("has spaces?", (" " in k))
print("len raw:", len(k), "len strip:", len(k.strip()))

repr: 'e000c20e40'
endswith newline? False
has spaces? False
len raw: 73 len strip: 73


In [3]:
import os, requests
headers = {"Authorization": f"Bearer {os.environ['OPENROUTER_API_KEY'].strip()}"}
r = requests.get("https://openrouter.ai/api/v1/models", headers=headers, timeout=20)
print(r.status_code)
print(r.text[:300])


200
{"data":[{"id":"anthropic/claude-sonnet-4.6","canonical_slug":"anthropic/claude-4.6-sonnet-20260217","hugging_face_id":"","name":"Anthropic: Claude Sonnet 4.6","created":1771342990,"description":"Sonnet 4.6 is Anthropic's most capable Sonnet-class model yet, with frontier performance across coding, 


## 1) Sanity check: run the Python execution tool

This runs locally with timeouts and basic restrictions. It is **not** a hardened sandbox.


In [4]:
from tools import run_python

code = """
print('hello from tool')
print(sum([1,2,3]))
"""

run_python(code)


{'ok': True,
 'stdout': 'hello from tool\n6\n',
 'stderr': '',
 'exit_code': 0,
 'note': 'Execution policy: temporary working directory, time-limited, and blocks some risky imports/calls. This is NOT a hardened sandbox.'}

## 2) Run a single agent task

We will run one end-to-end task:
- Planner proposes solution + code
- Executor runs code
- If fails, Fixer patches and retries (up to 3 attempts)


In [4]:
from agent_lib import run_task

task = "Write a Python function to compute Fibonacci(n) efficiently and print Fibonacci(35)."
result = run_task(task)

result['last_run']


=== GENERATED CODE ===
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b

print(fibonacci(35))


{'ok': True,
 'stdout': '9227465\n',
 'stderr': '',
 'exit_code': 0,
 'note': 'Execution policy: temporary working directory, time-limited, and blocks some risky imports/calls. This is NOT a hardened sandbox.'}

## 3) Workshop exercises

Try the tasks below. You can also author your own.
Tip: keep tasks self-contained and offline.


In [5]:
tasks = [
    "Parse this CSV string and compute the average of the 'latency_ms' column:\n\nts,latency_ms\n1,120\n2,110\n3,130\n4,90\n",
    "Implement rolling z-score anomaly score for this list and print the top 3 most anomalous points: [10,11,9,10,10,200,11,10,9,10]",
    "Given a list of (user_id, event_time, event_type), compute per-user session counts (30-min gap) and print a dict."
]

for t in tasks:
    print('\n' + '='*80)
    print('TASK:', t)
    out = run_task(t)
    print('OK:', out['last_run']['ok'])
    print('STDOUT:\n', out['last_run']['stdout'])
    if not out['last_run']['ok']:
        print('STDERR:\n', out['last_run']['stderr'])



TASK: Parse this CSV string and compute the average of the 'latency_ms' column:

ts,latency_ms
1,120
2,110
3,130
4,90

=== GENERATED CODE ===
import csv
from io import StringIO

csv_string = """ts,latency_ms
1,120
2,110
3,130
4,90"""

f = StringIO(csv_string)
reader = csv.DictReader(f)
latencies = [int(row['latency_ms']) for row in reader]
average_latency = sum(latencies) / len(latencies)
print(average_latency)
OK: True
STDOUT:
 112.5


TASK: Implement rolling z-score anomaly score for this list and print the top 3 most anomalous points: [10,11,9,10,10,200,11,10,9,10]
=== GENERATED CODE ===
import numpy as np

data = [10, 11, 9, 10, 10, 200, 11, 10, 9, 10]
window_size = 3
z_scores = []

for i in range(len(data)):
    if i < window_size - 1:
        z_scores.append(0)
    else:
        window = data[i - window_size + 1:i + 1]
        mean = np.mean(window)
        std = np.std(window)
        if std == 0:
            z = 0
        else:
            z = (data[i] - mean) / std
        z_

## 3a) Advanced: tighten/loosen execution policy

In `tools.py`, you can change:
- timeout
- banned patterns

For meetup safety, keep it restrictive.


## 4) Applied Exercise: Research Agent

Unlike the code-execution agent, this agent:
- Plans multi-step searches
- Gathers information from the web via Tavily
- Synthesizes findings into a structured report

**Use case:** competitive intelligence, market research, due diligence


In [6]:
from research_agent import run_research

question = "What are the top 3 AI chip companies in 2024 and what's their competitive advantage?"

print(f"RESEARCH QUESTION:\n{question}\n")

result = run_research(question)

print("\n" + "="*60)
print("FINAL REPORT:")
print("="*60)
print(result["report"])

RESEARCH QUESTION:
What are the top 3 AI chip companies in 2024 and what's their competitive advantage?


PLANNED QUERIES:
  1. top AI chip companies 2024 competitive advantage
  2. leading AI chip manufacturers 2024 market position
  3. AI chip industry leaders 2024 technology comparison

🔍 Searching: top AI chip companies 2024 competitive advantage
   → Found 3 sources
🔍 Searching: leading AI chip manufacturers 2024 market position
   → Found 3 sources
🔍 Searching: AI chip industry leaders 2024 technology comparison
   → Found 3 sources

✓ Collected 9 total sources


FINAL REPORT:
**Executive Summary**
NVIDIA, AMD, and Google (Alphabet) are the top three AI chip companies in 2024, each leveraging distinct competitive advantages in performance, cost-effectiveness, and specialized architectures. NVIDIA dominates with powerful GPUs for AI training and inference, AMD offers cost-effective alternatives, and Google leads in custom AI accelerators for its ecosystem.

**Key Findings**
- NVID

## 5) Optional: make it multi-agent

You can extend `agent_lib.py` into multiple agents:
- planner
- coder
- executor
- verifier

LangGraph makes these edges explicit.
